In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

In [ ]:
%load_ext autoreload
%autoreload 2

## Create DATASET

### 1 - Generate mask instances

In [ ]:
from data_utils import generate_annotations_json

In [ ]:
dataset_path = '/root/data/erko/'
mask_dir = 'instance_labels'
frame_dir = 'labeled_frames'
dataset_name = 'fish_only'
class_ID_path = '/root/data/erko/class_ID.csv'

In [ ]:
#d_train, d_test = generate_annotations_json(dataset_path=dataset_path,
#                          mask_dir=mask_dir,
#                          frame_dir=frame_dir,
#                          dataset_name=dataset_name,
#                          class_ID_path=class_ID_path,
#                        )

### 2 - Create folder structure

In [ ]:
from data_utils import create_training_folder_structure

In [ ]:
train_json = 'instance_fish_only_train.json'
test_json = 'instance_fish_only_test.json'
train_target_dir = 'instance_train_images'
test_target_dir = 'instance_test_images'


In [ ]:
#create_training_folder_structure(dataset_path=dataset_path,
#                                 train_json=train_json,
#                                 test_json=test_json,
#                                 train_target_dir=train_target_dir,
#                                 test_target_dir=test_target_dir, 
#                                 frame_dir=frame_dir)

### 3 - Datasets

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import keras

sys.path.insert(0, '../libraries')
from mrcnn.config import Config
import mrcnn.utils as utils
import mrcnn.model as modellib
import mrcnn.visualize as visualize
from mrcnn.model import log
import mcoco.coco as coco
# import mextra.utils as extra_utils
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

%matplotlib inline
%config IPCompleter.greedy=True
BASE_DIR = '/root/data/models/erko/mask_rcnn_instance_segmentation'
DATA_DIR = '/root/data/erko/'
WEIGHTS_DIR = os.path.join(BASE_DIR, "weights")
MODEL_DIR = os.path.join(BASE_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(WEIGHTS_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
# train dataset
dataset_train  = coco.CocoDataset()
dataset_train.load_coco('/root/data/aquabyte-images/cocofiles/coco_clear_full_curved_2018-09-07.json', class_ids=[0])
dataset_train.load_coco('/root/data/aquabyte-images/cocofiles/coco_clear_full_curved_2018-09-08.json', class_ids=[0])
# dataset_train.load_coco('/root/data/aquabyte-images/cocofiles/coco_clear_full_curved_2018-09-10.json', class_ids=[0])
# dataset_train.load_coco('/root/data/aquabyte-images/cocofiles/coco_clear_full_curved_2018-09-11.json', class_ids=[0])
dataset_train.prepare()

In [ ]:
# val dataset
dataset_validate  = coco.CocoDataset()
dataset_validate.load_coco('/root/data/aquabyte-images/cocofiles/coco_clear_full_curved_2018-09-07.json', class_ids=[0])
dataset_validate.load_coco('/root/data/aquabyte-images/cocofiles/coco_clear_full_curved_2018-09-08.json', class_ids=[0])
# dataset_train.load_coco('/root/data/aquabyte-images/cocofiles/coco_clear_full_curved_2018-09-10.json', class_ids=[0])
# dataset_train.load_coco('/root/data/aquabyte-images/cocofiles/coco_clear_full_curved_2018-09-11.json', class_ids=[0])
dataset_validate.prepare()

In [ ]:
# Load and display random samples# Load  
image_ids = np.random.choice(dataset_train.image_ids, 4)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

## Configuration

In [ ]:
image_size = 512
rpn_anchor_template = (1, 2, 4, 8, 16) # anchor sizes in pixels
rpn_anchor_scales = tuple(i * (image_size // 16) for i in rpn_anchor_template)

class FishConfig(Config):
    """Configuration for training on the shapes dataset.
    """
    # name your experiments here
    NAME = "clear_full"

    # Train on 1 GPU and 2 images per GPU. Put multiple images on each
    # GPU if the images are small. Batch size is 2 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + 3 shapes (triangles, circles, and squares)

    # Use smaller images for faster training. 
    IMAGE_MAX_DIM = image_size
    IMAGE_MIN_DIM = image_size
    
    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = rpn_anchor_scales

    # Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    STEPS_PER_EPOCH = 400

    VALIDATION_STEPS = STEPS_PER_EPOCH / 20
    
config = FishConfig()
config.display()

## Model

In [ ]:
model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)

In [ ]:
inititalize_weights_with = "coco"  # imagenet, coco, or last

if inititalize_weights_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
    
elif inititalize_weights_with == "coco":
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
    
elif inititalize_weights_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last()[1], by_name=True)

## Training

In [ ]:
# add more callbacks here if necessary
callbacks = [
    keras.callbacks.TensorBoard(log_dir=model.log_dir,
                                histogram_freq=0, write_graph=True, write_images=False),
    keras.callbacks.ModelCheckpoint(model.checkpoint_path,
                                    verbose=0, save_weights_only=True),
]

In [ ]:
model.train(dataset_train, dataset_validate, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=100, # starts from the previous epoch, so only 1 additional is trained 
            layers="all",
            callback_list=callbacks)

## Detection

### 1 - Inference config

In [ ]:
class InferenceConfig(FishConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
print(model.find_last()[1])
model_path = model.find_last()[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

### Test on random image from test set

In [ ]:
image_id = random.choice(dataset_validate.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_validate, inference_config, 
                           image_id, use_mini_mask=False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_validate.class_names, figsize=(8, 8))

results = model.detect([original_image], verbose=1)

r = results[0]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_validate.class_names, r['scores'], ax=get_ax())

## Evaluation

In [ ]:
predictions =\
extra_utils.compute_multiple_per_class_precision(model, inference_config, dataset_validate, iou_threshold=0.5)
complete_predictions = []

for shape in predictions:
    complete_predictions += predictions[shape]
    print("{} ({}): {}".format(shape, len(predictions[shape]), np.mean(predictions[shape])))

print("--------")
print("average: {}".format(np.mean(complete_predictions)))

## Testing

In [ ]:
image_ids = dataset_validate.image_ids

In [ ]:
image_id = image_ids[0]

In [ ]:
image_id

In [ ]:
image, _, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset_validate, inference_config,
                                image_id, use_mini_mask=False)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(image)

In [ ]:
plt.imshow(gt_mask[..., 0])

In [ ]:
results = model.detect([image], verbose=0)

In [ ]:
plt.imshow(results[0]['masks'][..., 2])